In [10]:
import json
import requests
import requests_cache 

requests_cache.install_cache(cache_name='discogs_cache', backend='sqlite', expire_after=86400)

# Input variables
label_id = "771357"
min_average = 4.7

In [11]:
# Define function to get label's releases
def get_label(label_id, page):
    # Variables
    label_url = "https://api.discogs.com/labels/{}/releases?page={}&per_page=25".format(label_id, page)

    # Make the HTTP request
    response = requests.get(label_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        response_dict = response.json()
        
        # Print the formatted JSON
        # print(json.dumps(response_dict, indent=4))

        # Return value
        return response_dict
    else:
        # Print an error message if the request was not successful
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")

# Define function to get label's releases recursively
def get_label_recursively(label_id, page, all_releases=None):
    # Variables
    label_url = "https://api.discogs.com/labels/{}/releases?page={}&per_page=25".format(label_id, page)

    # Initialize all_releases if not provided
    if all_releases is None:
        all_releases = {"pagination": {"page": 0, "pages": 0, "per_page": 25, "items": 0}, "releases": []}

    # Make the HTTP request
    response = requests.get(label_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        response_dict = response.json()

        # Print the formatted JSON
        # print(json.dumps(response_dict, indent=4))

        # Add releases to the aggregate dictionary
        all_releases["releases"].extend(response_dict["releases"])
        all_releases["pagination"] = response_dict["pagination"]

        # Check if there are more pages to fetch
        next_page = response_dict["pagination"]["urls"].get("next")
        if next_page:
            # Recursively call the function for the next page
            get_label_recursively(label_id, page + 1, all_releases)

    else:
        # Print an error message if the request was not successful
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")

    return all_releases

# Define function to get community release rating
def get_rating(release_url):

    # Variables
    rating_url = release_url + "/rating"
    
    # Make the HTTP request
    response = requests.get(rating_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        response_dict = response.json()

        # Print the formatted JSON
        # print(json.dumps(response_dict, indent=4))

        # Return value
        return response_dict
    else:
        # Print an error message if the request was not successful
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")

# Define function to remove duplicates from an input_list and key_path
def remove_duplicates(input_list, key_path):
    unique_list = []
    seen_ids = set()

    for item in input_list:
        item_id = get_nested_value(item, key_path)
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_list.append(item)

    return unique_list

# Handle nested keys
def get_nested_value(dictionary, key_path):
    current_value = dictionary
    for key in key_path:
        current_value = current_value.get(key, {})
    return current_value

# Define function to match releases with filtered_rating_list
def create_matching_releases(response_dict, filtered_rating_list):
    # Create a dictionary mapping release_id to rating_info
    rating_info_dict = {item["release_id"]: item["rating"] for item in filtered_rating_list}

    # # Debug
    # print("##### DEBUG #####")
    # print(type(rating_info_dict))
    # print(json.dumps(rating_info_dict, indent=4))

    # Extract information from response_dict only for matching release_ids
    matching_releases = [
        {
            "release_info": release_info,
            "rating_info": rating_info_dict.get(release_info["id"])
        }
        for release_info in response_dict["releases"] if release_info["id"] in rating_info_dict
    ]

    # # Debug
    # print("##### DEBUG #####")
    # print(type(matching_releases))
    # for item in matching_releases:
    #     print(json.dumps(item, indent=4))

    return matching_releases

# Define function to create output list
def create_output_list(unique_matching_list):
    output_list = []

    for release in unique_matching_list:
        new_dict = {
            "catno": release["release_info"]["catno"],
            "artist": release["release_info"]["artist"],
            "title": release["release_info"]["title"],
            "year": release["release_info"]["year"],
            "format": release["release_info"]["format"],
            "url": "https://www.discogs.com/es/release/{}".format(release["release_info"]["id"]),
            "rating": release["rating_info"]
        }
        output_list.append(new_dict)

    return output_list    

In [12]:
# Call get_label function for first time
# response_label_dict = get_label(label_id,1)
# print()
# print(type(response_dict))

# Call get_label_recursively function
response_label_dict = get_label_recursively(label_id,1)

# Debug
print("##### DEBUG #####")
print(type(response_label_dict))
print(json.dumps(response_label_dict, indent=4))

##### DEBUG #####
<class 'dict'>
{
    "pagination": {
        "page": 6,
        "pages": 6,
        "per_page": 25,
        "items": 137,
        "urls": {
            "first": "https://api.discogs.com/labels/771357/releases?page=1&per_page=25",
            "prev": "https://api.discogs.com/labels/771357/releases?page=5&per_page=25"
        }
    },
    "releases": [
        {
            "status": "Accepted",
            "format": "2x12\", Comp",
            "catno": "TRP006",
            "thumb": "",
            "resource_url": "https://api.discogs.com/releases/8544555",
            "title": "When I Was 14",
            "id": 8544555,
            "year": 2016,
            "artist": "Various"
        },
        {
            "status": "Accepted",
            "format": "2x12\", Comp",
            "catno": "TRP006",
            "thumb": "",
            "resource_url": "https://api.discogs.com/releases/8544555",
            "title": "When I Was 14",
            "id": 8544555,
          

In [4]:
# Get community release rating
response_rating_list = []
for release in response_label_dict["releases"]:
    release_url = release["resource_url"]
    response_rating_dict = get_rating(release_url)
    response_rating_list.append(response_rating_dict) 

# Debug
print("##### DEBUG #####")
print("length response_rating_list: " + str(len(response_rating_list)))
for item in response_rating_list:
   print(item) 

##### DEBUG #####
length response_rating_list: 37
{'release_id': 24998860, 'rating': {'count': 2, 'average': 4.5}}
{'release_id': 26600165, 'rating': {'count': 2, 'average': 5.0}}
{'release_id': 10446578, 'rating': {'count': 41, 'average': 4.49}}
{'release_id': 10530622, 'rating': {'count': 193, 'average': 4.58}}
{'release_id': 11058723, 'rating': {'count': 57, 'average': 4.56}}
{'release_id': 26606444, 'rating': {'count': 3, 'average': 4.33}}
{'release_id': 11184384, 'rating': {'count': 33, 'average': 4.45}}
{'release_id': 13217748, 'rating': {'count': 26, 'average': 4.27}}
{'release_id': 15163690, 'rating': {'count': 4, 'average': 4.5}}
{'release_id': 14630885, 'rating': {'count': 1, 'average': 3.0}}
{'release_id': 13862589, 'rating': {'count': 21, 'average': 4.38}}
{'release_id': 14048028, 'rating': {'count': 60, 'average': 4.43}}
{'release_id': 14298828, 'rating': {'count': 49, 'average': 4.63}}
{'release_id': 15019933, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 1465164

In [5]:
# Remove duplicates from "response_rating_list" based on "release_id" key
unique_rating_list = remove_duplicates(response_rating_list, ["release_id"])

# Debug
print("##### DEBUG #####")
print("length unique_rating_list: " + str(len(unique_rating_list)))
for item in unique_rating_list:
   print(item) 

##### DEBUG #####
length unique_rating_list: 37
{'release_id': 24998860, 'rating': {'count': 2, 'average': 4.5}}
{'release_id': 26600165, 'rating': {'count': 2, 'average': 5.0}}
{'release_id': 10446578, 'rating': {'count': 41, 'average': 4.49}}
{'release_id': 10530622, 'rating': {'count': 193, 'average': 4.58}}
{'release_id': 11058723, 'rating': {'count': 57, 'average': 4.56}}
{'release_id': 26606444, 'rating': {'count': 3, 'average': 4.33}}
{'release_id': 11184384, 'rating': {'count': 33, 'average': 4.45}}
{'release_id': 13217748, 'rating': {'count': 26, 'average': 4.27}}
{'release_id': 15163690, 'rating': {'count': 4, 'average': 4.5}}
{'release_id': 14630885, 'rating': {'count': 1, 'average': 3.0}}
{'release_id': 13862589, 'rating': {'count': 21, 'average': 4.38}}
{'release_id': 14048028, 'rating': {'count': 60, 'average': 4.43}}
{'release_id': 14298828, 'rating': {'count': 49, 'average': 4.63}}
{'release_id': 15019933, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 14651640,

In [6]:
# Filter releases with average >= min_average
filtered_rating_list = [item for item in unique_rating_list if item["rating"]["average"] >= min_average]

# Debug
print("##### DEBUG #####")
for item in filtered_rating_list:
    print(item)

##### DEBUG #####
{'release_id': 26600165, 'rating': {'count': 2, 'average': 5.0}}
{'release_id': 15019933, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 16270494, 'rating': {'count': 34, 'average': 4.79}}
{'release_id': 17372029, 'rating': {'count': 53, 'average': 4.74}}
{'release_id': 24520745, 'rating': {'count': 7, 'average': 5.0}}
{'release_id': 24700613, 'rating': {'count': 2, 'average': 5.0}}
{'release_id': 25237264, 'rating': {'count': 18, 'average': 4.72}}
{'release_id': 23283611, 'rating': {'count': 23, 'average': 4.7}}


In [7]:
# Create a list with the matching releases
matching_releases_list = create_matching_releases(response_label_dict, filtered_rating_list)

# Debug
print("##### DEBUG #####")
print("length matching_releases: " + str(len(matching_releases_list)))
for item in matching_releases_list:
   print(json.dumps(item, indent=4)) 

##### DEBUG #####
length matching_releases: 8
{
    "release_info": {
        "status": "Accepted",
        "format": "5xFile, MP3, EP, 320",
        "catno": "ARTSCORE 016",
        "thumb": "",
        "resource_url": "https://api.discogs.com/releases/26600165",
        "title": "The Dark Welcome",
        "id": 26600165,
        "year": 2022,
        "artist": "Tensal"
    },
    "rating_info": {
        "count": 2,
        "average": 5.0
    }
}
{
    "release_info": {
        "status": "Accepted",
        "format": "5xFile, WAV",
        "catno": "ARTSCORE008",
        "thumb": "",
        "resource_url": "https://api.discogs.com/releases/15019933",
        "title": "Die Illusion",
        "id": 15019933,
        "year": 2019,
        "artist": "Obscure Shape & SHDW*"
    },
    "rating_info": {
        "count": 1,
        "average": 5.0
    }
}
{
    "release_info": {
        "status": "Accepted",
        "format": "12\"",
        "catno": "ARTSCORE013",
        "thumb": "",
    

In [8]:
# Remove duplicates from matching_releases_list based on the "id" key
unique_matching_list = remove_duplicates(matching_releases_list, ["release_info", "id"])

# Debug
print("##### DEBUG #####")
print("length unique_matching_list: " + str(len(unique_matching_list)))
for release in unique_matching_list:
    print("Release Info:")
    print(json.dumps(release["release_info"], indent=4))
    print("Rating Info:")
    print(json.dumps(release["rating_info"], indent=4))
    print()

##### DEBUG #####
length unique_matching_list: 8
Release Info:
{
    "status": "Accepted",
    "format": "5xFile, MP3, EP, 320",
    "catno": "ARTSCORE 016",
    "thumb": "",
    "resource_url": "https://api.discogs.com/releases/26600165",
    "title": "The Dark Welcome",
    "id": 26600165,
    "year": 2022,
    "artist": "Tensal"
}
Rating Info:
{
    "count": 2,
    "average": 5.0
}

Release Info:
{
    "status": "Accepted",
    "format": "5xFile, WAV",
    "catno": "ARTSCORE008",
    "thumb": "",
    "resource_url": "https://api.discogs.com/releases/15019933",
    "title": "Die Illusion",
    "id": 15019933,
    "year": 2019,
    "artist": "Obscure Shape & SHDW*"
}
Rating Info:
{
    "count": 1,
    "average": 5.0
}

Release Info:
{
    "status": "Accepted",
    "format": "12\"",
    "catno": "ARTSCORE013",
    "thumb": "",
    "resource_url": "https://api.discogs.com/releases/16270494",
    "title": "REM",
    "id": 16270494,
    "year": 2020,
    "artist": "Introversion (3)"
}
Rat

In [9]:
# Create specific output
output_list = create_output_list(unique_matching_list)

# print(output_list)
print(json.dumps(output_list, indent=4))

[
    {
        "catno": "ARTSCORE 016",
        "artist": "Tensal",
        "title": "The Dark Welcome",
        "year": 2022,
        "format": "5xFile, MP3, EP, 320",
        "url": "https://www.discogs.com/es/release/26600165",
        "rating": {
            "count": 2,
            "average": 5.0
        }
    },
    {
        "catno": "ARTSCORE008",
        "artist": "Obscure Shape & SHDW*",
        "title": "Die Illusion",
        "year": 2019,
        "format": "5xFile, WAV",
        "url": "https://www.discogs.com/es/release/15019933",
        "rating": {
            "count": 1,
            "average": 5.0
        }
    },
    {
        "catno": "ARTSCORE013",
        "artist": "Introversion (3)",
        "title": "REM",
        "year": 2020,
        "format": "12\"",
        "url": "https://www.discogs.com/es/release/16270494",
        "rating": {
            "count": 34,
            "average": 4.79
        }
    },
    {
        "catno": "ARTSCORE014",
        "artist": "Yan 